In [259]:
import numpy as np
import cv2
from itertools import cycle
from collections import deque
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from pathlib import Path
from tqdm import tqdm
import os

In [128]:
STEP      = 5          # grid step (pixels)
EIG_MULT = 1e-3       # constant in Eq.(1)
TRACK_LEN   = 15         # L in the paper (Eq. 3)
JUMP_FRAC   = 0.7        # drop if a single step ≥ 70 % of total path
MIN_TOTAL   = 1.0        # drop static paths (< 1 px overall motion)

FLOW_KW     = dict(pyr_scale=0.5, levels=3, winsize=15,
                   iterations=3, poly_n=5, poly_sigma=1.2, flags=0)

In [129]:
def sample_dense_points(gray: np.ndarray,
                        step: int = 5,
                        eig_mult: float = 1e-3) -> np.ndarray:
    """
    Return an (N,2) int32 array of good seed points for one frame.
    Implements Eq.(1): reject texture-less pixels.
    """
    H, W = gray.shape
    ys, xs = np.mgrid[0:H:step, 0:W:step]           # regular grid
    grid   = np.vstack((xs.ravel(), ys.ravel())).T  # (N,2)

    saliency = cv2.cornerMinEigenVal(gray, 3, 3)    # Shi–Tomasi λ_min
    thresh   = eig_mult * saliency.max()
    keep     = saliency[grid[:, 1], grid[:, 0]] >= thresh
    return grid[keep].astype(np.float32)            # (x,y) float for sub-pixel

def compute_flow(prev, curr):

    return cv2.calcOpticalFlowFarneback(
        prev, curr, None,
        0.5,
        3,
        15,
        3,
        5, 1.2, 0
    )

In [162]:
def advance_tracks(tracks, flow,
                   jump_frac=0.7,
                   track_len=15,
                   min_total=0.5):
    """
    Move every active track one step forward.
    Returns (new_active, finished_list)
    finished_list items are length-track_len lists of (x,y).
    """
    H, W, _ = flow.shape
    new_active, finished = [], []

    for tr in tracks:
        #print(tr)
        x, y = tr[-1]
        dx, dy = flow[int(y), int(x)]
        nx, ny = x + dx, y + dy

        # out-of-frame?
        if not (0 <= nx < W and 0 <= ny < H):
            continue

        # sanity-check
        
        total_disp = np.hypot(nx - tr[0][0], ny - tr[0][1])
        #print(f'total_disp from the start of the trajectory:{total_disp}')
        #print(f'Current displacement: {np.hypot(dx, dy)}')
        if len(tr) > 1:
            if np.hypot(dx, dy) > jump_frac * (total_disp + 1e-8):
                #print('dropping trajectory due to high displacement.')
                continue                              # drop erratic

        tr.append((nx, ny))
        if len(tr) == track_len:
            if total_disp > min_total:
                finished.append(tr)
        else:
            new_active.append(tr)

    return new_active, finished

In [163]:
#The paper wants one active point per STEP × STEP grid cell at all times.
def reseed(tracks, gray, step):
    """
    Fill every step×step cell not already occupied by a track head.
    Modifies 'tracks' in-place by appending [seed] lists.
    """
    seeds = sample_dense_points(gray, step)
    if tracks:
        heads = np.array([tr[-1] for tr in tracks])
        dist  = np.linalg.norm(seeds[:, None] - heads[None], axis=2)
        seeds = seeds[dist.min(axis=1) >= step / 2] #If the cell already has a head, you discard that seed — 
                 #it’s already covered.If no head is in that cell, you keep the seed and start a new track.
        #print(seeds)
    tracks.extend([tuple(p)] for p in seeds)

In [164]:
def trajectory_shape(track):
    """30-D normalised displacement vector (Eq. 3)."""
    disp = np.diff(np.array(track), axis=0)          # (15,2)
    norm = np.linalg.norm(disp, axis=1).sum() + 1e-8
    return (disp / norm).flatten().astype(np.float32)


In [165]:
def extract_dense_trajectories(video_path,
                               step: int = 5,
                               track_len: int = 16,
                               return_tracks: bool = False):
    """
    Track dense points and return trajectory-shape descriptors.

    Parameters
    ----------
    video_path   : str / Path
    step         : grid spacing  (pixels)
    track_len    : #positions per trajectory (16 → 15 displacements = 30-D)
    return_tracks: if True → also return {frame_idx: [...] } dictionaries

    Returns
    -------
    If return_tracks == False:
        desc30 : (N, 30) float32 array
    If return_tracks == True:
        finished_by_frame : dict[int, list[list[(x,y)]]]
        shapes_by_frame   : dict[int, list[np.ndarray shape (30,)]]
        desc30            : (N, 30) float32 array
    """
    cap = cv2.VideoCapture(str(video_path))
    ok, frame = cap.read()
    if not ok:
        raise IOError(f"Cannot open {video_path}")

    prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # ── containers ──────────────────────────────────────────────────────
    tracks        = []       # active, still-growing lists of (x,y)
    descriptors   = []       # 30-D vectors (all finished)
    finished_dict = {}       # key = end frame idx,   value = list of tracks
    shape_dict    = {}       # key = end frame idx,   value = list of shape30

    # ── seed frame 0 ───────────────────────────────────────────────────
    tracks.extend([tuple(p)] for p in sample_dense_points(prev_gray, step))

    frame_idx = 0            # keeps track of *prev* frame’s index
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # A. Farnebäck flow prev → curr
        flow = compute_flow(prev_gray, gray)

        # B. advance tracks, collect those that hit length = track_len
        tracks, done = advance_tracks(tracks, flow, track_len=track_len)

        for tr in done:
            shape30 = trajectory_shape(tr)
            descriptors.append(shape30)

            if return_tracks:                       # store for Task-2
                finished_dict.setdefault(frame_idx, []).append(tr)
                shape_dict   .setdefault(frame_idx, []).append(shape30)

        # C. reseed empty grid cells on current frame
        reseed(tracks, gray, step)

        # slide window
        prev_gray = gray
        frame_idx += 1

    cap.release()

    desc30 = (np.vstack(descriptors)
              if descriptors else np.empty((0, 30), np.float32))

    if return_tracks:
        return finished_dict, shape_dict, desc30
    else:
        return desc30

In [167]:
finished_dict, shape_dict, desc30 = extract_dense_trajectories("datasets/dummy.avi", return_tracks=True)

In [168]:
desc30.shape

(146, 30)

In [172]:
len(finished_dict)

58

In [111]:
# pick six bright colours and recycle them
_colour_cycle = cycle([(255,0,0), (0,255,0), (0,0,255),
                       (255,255,0), (255,0,255), (0,255,255)])

def draw_done_paths(frame, done, colours=None, thickness=1):
    """
    frame    : BGR image you want to paint on (will be *modified* in-place)
    done     : list of finished trajectories; each is [(x0,y0), …, (x15,y15)]
    colours  : optional iterable of BGR tuples; will be recycled
    thickness: line width in pixels
    """
    if colours is None:
        colours = _colour_cycle
    for tr, col in zip(done, colours):
        for p1, p2 in zip(tr[:-1], tr[1:]):         # draw segment chain
            cv2.line(frame,
                     (int(p1[0]), int(p1[1])),
                     (int(p2[0]), int(p2[1])),
                     col, thickness, cv2.LINE_AA)
    return frame

In [112]:
cap = cv2.VideoCapture("datasets/dummy.avi")
_, first = cap.read()
canvas = first.copy()

for tr in done:          # list you kept during extraction
    draw_done_paths(canvas, [tr])     # wrap tr in list so API matches
cv2.imwrite("all_finished_paths.png", canvas)

True

In [192]:
PATCH = 32
HALF = PATCH//2

def crop_volume(frames, flows_u, flows_v, traj):

    L = 15
    H, W = frames[0].shape

    img_vol = np.zeros((L, PATCH, PATCH), np.float32)
    u_vol   = np.zeros_like(img_vol)
    v_vol   = np.zeros_like(img_vol)

    #fill each time dim (15) with the 32x32 patch centered on the trajectory head
    #anything that falls outside the image stays zero

    for t, (x, y) in enumerate(traj):
        x, y = int(round(x)), int(round(y)) #flow tracking gave us float, but to slice the arrays we need int

        #Original full frame bounds of the patch
        x0, x1 = x - HALF, x + HALF #left-right bounds
        y0, y1 = y - HALF, y + HALF #top-bottom bounds

        #Clip these bounds so they stay inside the image
        xs0, xs1 = max(0, x0), min(W, x1)
        ys0, ys1 = max(0, y0), min(H, y1)

        #The patches might bet clipped to not be 32x32 anymore, if they out of image bounds
        #Calulcate the offset for zero padding (the arrays we fill are already full of zeros)
        #So we only fill in those places which have a value from the frame

        dx0, dx1 = xs0-x0, PATCH - (x1 - xs1)
        dy0, dy1 = ys0-y0, PATCH - (y1 - ys1)

        img_vol[t, dy0:dy1, dx0:dx1] = frames[t][ys0:ys1, xs0:xs1]
        u_vol  [t, dy0:dy1, dx0:dx1] = flows_u[t][ys0:ys1, xs0:xs1]
        v_vol  [t, dy0:dy1, dx0:dx1] = flows_v[t][ys0:ys1, xs0:xs1]

        return img_vol, u_vol, v_vol
        
        

        

In [193]:
def tube_generate(vol):

    for ti in range(3):
        t0, t1 = ti*5, (ti+1)*5
        for ry in range(2):
            r0, r1 = ry*16, (ry+1)*16
            for cx in range(2):
                c0, c1 = cx*16, (cx+1)*16
                yield (t0, t1, r0, r1, c0, c1)

In [194]:
# fixed bin edges
BIN_HOG = 8
BIN_HOF = 9          # 8 orient + 1 static
BIN_MBH = 8

def hog_hist(patch):
    gx = cv2.Sobel(patch, cv2.CV_32F, 1, 0, ksize=1)
    gy = cv2.Sobel(patch, cv2.CV_32F, 0, 1, ksize=1)
    mag, ang = cv2.cartToPolar(gx, gy, angleInDegrees=True)
    bins = np.int32(ang // (360/BIN_HOG)) % BIN_HOG
    hist = np.bincount(bins.ravel(), mag.ravel(), BIN_HOG).astype(np.float32)
    return hist / (np.linalg.norm(hist)+1e-6)

def hof_hist(u_patch, v_patch, static_thr=1.0):
    mag, ang = cv2.cartToPolar(u_patch, v_patch, angleInDegrees=True)
    static = mag < static_thr
    bins = np.int32(ang // (360/8)) % 8
    bins[static] = 8                      # 9th bin
    hist = np.bincount(bins.ravel(), mag.ravel(), BIN_HOF).astype(np.float32)
    return hist / (np.linalg.norm(hist)+1e-6)

def mbh_hist(comp_patch):
    # comp_patch is u or v flow channel
    gx = cv2.Sobel(comp_patch, cv2.CV_32F, 1, 0, ksize=1)
    gy = cv2.Sobel(comp_patch, cv2.CV_32F, 0, 1, ksize=1)
    mag, ang = cv2.cartToPolar(gx, gy, angleInDegrees=True)
    bins = np.int32(ang // (360/BIN_MBH)) % BIN_MBH
    hist = np.bincount(bins.ravel(), mag.ravel(), BIN_MBH).astype(np.float32)
    return hist / (np.linalg.norm(hist)+1e-6)

In [195]:
def tube_descriptors(img_vol, u_vol, v_vol):
    HOG, HOF, MBHx, MBHy = [], [], [], []

    for t0,t1,r0,r1,c0,c1 in tube_generate(img_vol):
        # slice each 16×16×5 tube
        img_patch = img_vol[t0:t1, r0:r1, c0:c1]
        u_patch   = u_vol  [t0:t1, r0:r1, c0:c1]
        v_patch   = v_vol  [t0:t1, r0:r1, c0:c1]

        HOG .append(hog_hist(img_patch))
        HOF .append(hof_hist(u_patch, v_patch))
        MBHx.append(mbh_hist(u_patch))
        MBHy.append(mbh_hist(v_patch))

    return (np.hstack(HOG),  np.hstack(HOF),
            np.hstack(MBHx), np.hstack(MBHy))      # 96,108,96,96

In [196]:
def descriptor_426(frames, flows_u, flows_v, traj, shape30):
    img_vol, u_vol, v_vol = crop_volume(frames, flows_u, flows_v, traj)
    h_hog, h_hof, h_mbx, h_mby = tube_descriptors(img_vol, u_vol, v_vol)
    return np.hstack((shape30, h_hog, h_hof, h_mbx, h_mby))   # 30+96+108+96+96

# Task 2: Compute histograms

In [241]:
def compute_desc426(video):

    finished_dict, shape_dict, desc30 = extract_dense_trajectories(video, return_tracks=True)
    
    cap = cv2.VideoCapture(video)
    ok, prev = cap.read()
    prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    frames_buf, flows_u_buf, flows_v_buf = deque(maxlen=TRACK_LEN), deque(maxlen=TRACK_LEN), deque(maxlen=TRACK_LEN)

    desc426 = []
    frame_idx = 0
    while ok:
        gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
        frames_buf.append(gray)
    
        ok, frame = cap.read()
        if not ok:
            break
        flow = cv2.calcOpticalFlowFarneback(gray, cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY),
                                            None, 0.5, 3, 15, 3, 5, 1.2, 0)
        flows_u_buf.append(flow[...,0]); flows_v_buf.append(flow[...,1])
    
        # when a trajectory ends on *this* frame, build its 426-D descriptor
        for tr, shape30 in zip(finished_dict.get(frame_idx, []), shape_dict.get(frame_idx, [])):
            if len(frames_buf) == 15:  # buffers full
                d426 = descriptor_426(list(frames_buf),
                                      list(flows_u_buf),
                                      list(flows_v_buf),
                                      tr, shape30)
                desc426.append(d426)
    
        prev = frame
        frame_idx += 1
    
    cap.release()
    desc426 = np.array(desc426, np.float32)

    return desc426

# Task 3

In [221]:
def fit_pca(desc426_stack, n_comp=64):
    pca = PCA(n_components=n_comp, svd_solver='randomized', whiten=True)
    pca.fit(desc426_stack)              # train on all 426-D descriptors
    return pca

def project_pca(pca, desc426):
    return pca.transform(desc426).astype(np.float32)   # (N, 64)

In [222]:
def fit_gmm(desc64_stack, n_comp=4):
    gmm = GaussianMixture(n_components=n_comp,
                          covariance_type='diag',
                          max_iter=100, verbose=1)
    gmm.fit(desc64_stack)
    return gmm

In [223]:
def fisher_vector(desc64, gmm):
    """
    desc64 : (N_i , 64) array of one video
    Returns : (2*K*64,) FV with signed square-root + L2 norm
    """
    Q = gmm.predict_proba(desc64)          # (N_i, K) posteriors
    N = desc64.shape[0]

    # Compute first-order (means)
    diff = desc64[:, None, :] - gmm.means_  # (N_i,K,64)
    sigma = np.sqrt(gmm.covariances_)       # diag σ
    diff /= sigma                           # normalize
    S1 = (Q[..., None] * diff).sum(axis=0) / np.sqrt(gmm.weights_)[:, None]

    # Second-order (variances)
    diff2 = (diff**2 - 1)
    S2 = (Q[..., None] * diff2).sum(axis=0) / np.sqrt(2*gmm.weights_)[:, None]

    fv = np.hstack((S1.flatten(), S2.flatten())).astype(np.float32)

    # power- & L2-norm
    fv = np.sign(fv) * np.sqrt(np.abs(fv))
    fv /= (np.linalg.norm(fv) + 1e-8)
    return fv


In [252]:
def video_fv(descriptors426, pca, gmm):
    desc64 = project_pca(pca, descriptors426)   # (N,64)
    return fisher_vector(desc64, gmm)           # (2*K*64,)

In [225]:
def train_svm(X_train, y_train, C=1.0):
    clf = LinearSVC(C=C)
    clf.fit(X_train, y_train)
    return clf

def evaluate(clf, X_test, y_test):
    pred = clf.predict(X_test)
    acc  = accuracy_score(y_test, pred)
    return acc

# Task 4:

In [213]:
CACHE_FV = Path("fv_cache")      # one .npy per video
CACHE_FV.mkdir(exist_ok=True)

In [237]:
def read_split(txt):
    vids, labels = [], []
    for line in Path(txt).read_text().splitlines():
        fname, lab = line.strip().split()
        vids.append(os.path.join(os.getcwd(),f'datasets/UCF-3/{fname}'))
        labels.append(int(lab))
    return vids, np.array(labels, np.int32)

In [238]:
print("Loading train / test file lists …")
dataset_dir = os.path.join(os.getcwd(), 'datasets/UCF-3')
print(dataset_dir)
train_vids, y_train = read_split(os.path.join(dataset_dir, 'train.txt'))
test_vids,  y_test  = read_split(os.path.join(dataset_dir, 'test.txt'))

Loading train / test file lists …
/Users/gauravniranjan/Documents/Video Analytics/Assignments/Ex1/datasets/UCF-3


In [243]:
print("Gathering 426-D descriptors to fit PCA …")
all_desc = []
for vid in tqdm(train_vids):
    desc426 = compute_desc426(vid)         # Task-2 output
    all_desc.append(desc426)
all_desc = np.vstack(all_desc)                 # shape (M, 426)

Gathering 426-D descriptors to fit PCA …


100%|████████████████████████████████████████████████| 105/105 [1:12:42<00:00, 41.54s/it]


In [244]:
print("Fitting 64-D PCA on", all_desc.shape[0], "train descriptors …")
pca = fit_pca(all_desc, n_comp=64)
desc64 = pca.transform(all_desc)

Fitting 64-D PCA on 1386449 train descriptors …


In [245]:
print("GMM …")
gmm = fit_gmm(desc64, n_comp=5)

GMM …
Initialization 0
  Iteration 10
  Iteration 20
Initialization converged.


In [253]:
def fv_for_video(vid):
    fv_file = CACHE_FV / (Path(vid).stem + ".npy")
    if fv_file.exists():
        return np.load(fv_file)
    # else compute from scratch
    desc426 = compute_desc426(vid)
    fv = video_fv(desc426, pca, gmm)           # returns (FVdim,)
    np.save(fv_file, fv)
    return fv

In [ ]:
print("Building Fisher vectors …")
X_train = np.vstack([fv_for_video(v) for v in tqdm(train_vids)])
X_test  = np.vstack([fv_for_video(v) for v in tqdm(test_vids)])

In [255]:
print("Feature dims:", X_train.shape, X_test.shape)

Feature dims: (105, 640) (45, 640)


In [256]:
clf = LinearSVC(C=1.0, dual=False)             # dual=False for high-dim data
clf.fit(X_train, y_train)

LinearSVC(dual=False)

In [261]:
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {acc*100:.2f}%")

cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)


Test accuracy: 93.33%
Confusion matrix:
 [[15  0  0]
 [ 0 12  3]
 [ 0  0 15]]
